In [46]:
from matplotlib import pyplot as plt
%matplotlib inline

In [104]:
import os,sys
import numpy as np
import pandas as pd
# import xgboost as xgb

# Загрузим данные
 * При первом запуске скачаем train.csv и evaluation.csv
 * При повторных запусках файлы уже будут на месте

In [2]:
data = pd.DataFrame.from_csv("./train.csv")

In [22]:
data.head(1)

,cancel_time,dist,driver_found,due,f_class,lat,lon,s_class,t_class,burned
60,55,17990.125431,False,2014-01-01 00:09:32.000,econom,55.75013,37.823242,NaN,NaN,True


In [100]:
len(data.loc[data['cancel_time']<500])*1./ len(data) 

0.9578409204512395

In [101]:
data = data.loc[data['cancel_time']<500]

# попредсказываем cancel_time

In [102]:
# plt.figure(figsize=(16, 8))
# trash = plt.hist( data.loc[(data['burned']==1) & (data['cancel_time']<100000)]['cancel_time'], bins=1000 )
# plt.ylim(0, 5000)
# plt.xlim(0, 100000)

In [52]:
# data.loc[(data['burned']==0)]['cancel_time'].value_counts()

In [51]:
# data.loc[(data['burned']==1)]['cancel_time'].value_counts()

In [3]:
test = pd.DataFrame.from_csv("./test.csv")

In [21]:
test.head(1)

,dist,due,f_class,lat,lon,s_class,t_class
Ids,,,,,,,
0,1748.201178,2014-04-19 18:15:00.000,econom,55.731713,37.487353,business,vip


In [7]:
X = data[data.columns[:len(data.columns)-1]]

# citycenters

In [ ]:
#!!!!!
X = pd.DataFrame.from_csv("./train_with_cityid.csv")

In [8]:
centers = [
    [55.754216, 37.61343],  #0
    [51.661535, 39.200287], #1
    [56.326887, 44.005986], #2
    [55.798551, 49.106324], #3
    [59.939095, 30.315868]  #4
]

from geopy.distance import vincenty

minminmin = 10**15

def city_id(coords):
    mn = 0
    min_value = minminmin
    for i in range(len(centers)):
        
        dist = vincenty(coords, centers[i]).km
        if (dist < min_value):
            min_value = dist
            mn = i
    if (min_value > 10**3):
        return -1
    return mn

In [9]:
XV = X.values
X['city_id'] = [city_id([x[5], x[6]]) for x in XV]

In [17]:
X.to_csv('train_with_cityid.csv')

In [110]:
X = pd.read_csv('train_with_cityid.csv')

In [115]:
testV = test.values
test['city_id'] = [city_id([x[3], x[4]]) for x in testV]

In [116]:
test.to_csv('test_with_cityid.csv')

In [117]:
test = pd.read_csv('test_with_cityid.csv')

# concat

In [15]:
x_col = ['dist', 'due', 'lat', 'lon', 'f_class', 's_class', 't_class', 'city_id']
# X_and_test = pd.concat([X_copy[x_col], test_copy[x_col]])

# Preprocessing

In [111]:
#Используем данные о праздниках отсюда - http://eduscan.net/help/calendar2014 http://newslab.ru/article/559455

holidays = "1.01,2.01,3.01,4.01,5.01,6.01,7.01,8.01,"\
"23.02,8.03,9.03,10.03,1.05,2.05,3.05,4.05,9.05,10.05,11.05,12.06,13.06".split(',')

holidays = map(lambda s: tuple(map(int,s.split('.'))),holidays)

#вытаскиватель категориальных фичей
from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer(sparse=False,dtype=np.bool)

# используем встроенный питоновый парсер времени
from dateutil import parser

def preprocess_data(X_raw):

    #распарсим время
    datetimes = list(X_raw.due.apply(lambda x: parser.parse(x)))

    #время дня от 0 до 1
    rel_times = map(lambda dt: (dt.hour*60 + dt.minute)/(24.*60), datetimes)

    #день недели
    week_days = map( lambda x: x.isoweekday() , datetimes)

    rel_times = np.array(rel_times)
    week_days = np.array(week_days)
    
    is_holiday = map(lambda dt: (dt.day,dt.month) in holidays,
                 datetimes)

    dow_names = ['mon','tue','wed','thu','fri','sat','sun']
    
    city_names = [0, 1, 2, 3, 4, 5]
    
    data_dict = ( {'city_id':city_names[city_id+1], 
                   'f_class':f,
                   's_class':(s if (pd.isnull(s)) else (str(f) + str(s))),
                   't_class':(t if (pd.isnull(t)) else(str(f) + str(s) + str(t))),
                   'day_of_week':dow_names[day_of_week-1]}
                 
            for (city_id,f,s,t),day_of_week in zip(X_raw[['city_id', 
                                                         'f_class',
                                                         's_class',
                                                         't_class']].values, week_days) )

    
    Xcat = vectorizer.fit_transform(data_dict)

    other_features = ["dist","lat","lon"]
    
    Xreal = X_raw[other_features].values
    
    
    Xfull = np.concatenate([
            Xreal,
            Xcat            
        ],axis=1)
    
    Xfull = pd.DataFrame(Xfull,columns=other_features+vectorizer.feature_names_)
    
    Xfull["time_of_day_rel"] = rel_times
    Xfull["is_holiday"] = is_holiday

    
    return Xfull

In [112]:
X = preprocess_data(X)

In [113]:
X.to_csv('train_preprocessed.csv')

In [114]:
X.columns

Index([u'dist', u'lat', u'lon', u'city_id', u'day_of_week=fri',
       u'day_of_week=mon', u'day_of_week=sat', u'day_of_week=sun',
       u'day_of_week=thu', u'day_of_week=tue', u'day_of_week=wed', u'f_class',
       u'f_class=business', u'f_class=econom', u'f_class=vip', u's_class',
       u's_class=businesseconom', u's_class=businessvip',
       u's_class=econombusiness', u's_class=economvip', u's_class=vipbusiness',
       u's_class=vipeconom', u't_class', u't_class=businesseconomvip',
       u't_class=businessvipeconom', u't_class=econombusinessvip',
       u't_class=economvipbusiness', u't_class=vipbusinesseconom',
       u't_class=vipeconombusiness', u'time_of_day_rel', u'is_holiday'],
      dtype='object')

In [ ]:
# y = data['burned'].values
# X_NEW = X_and_test_copy.head(len(y))
# TEST_NEW = X_and_test_copy[len(y):]

In [ ]:
# X_NEW_copy = X_NEW.copy()
# TEST_NEW_copy = TEST_NEW.copy()

In [ ]:
# print X_NEW_copy.shape, y.shape

# learning

In [ ]:
# %%time

from sklearn.ensemble import RandomForestClassifier

#Также обучим более "случайную" версию RandomForest
model = RandomForestClassifier(n_estimators=150,n_jobs=-1,min_samples_split=75,min_samples_leaf=20)

# model.fit(Xtr,Ytr)
model.fit(X_NEW, y)




In [ ]:
from sklearn.metrics import roc_auc_score,accuracy_score,precision_score,recall_score
from sklearn.metrics import roc_curve


for source_i, Xi,Yi in [
                            ["train",Xtr,Ytr],
                            ["val",Xval,Yval]
                                ]:
    
    # Предскажем вероятность сгорания
    Yi_pred_proba = model.predict_proba(Xi)[:,1]
    
    #Поделим предсказание на сгоревшие и не сгоревшие
    Yi_pred_class = np.argsort(Yi_pred_proba) < 10000
    
    auc = roc_auc_score(Yi,Yi_pred_proba)
    acc = accuracy_score(Yi,Yi_pred_class)
    
    print '%s: \t AUC = %.5f \t Accuracy = %.5f'%(source_i, auc, acc)
    
    fpr,tpr,_ = roc_curve(Yi,Yi_pred_proba)
    
    plt.plot(fpr,tpr,label = source_i)
    
plt.legend()


In [ ]:
# from datanight import plot_forest_feature_importances
# plot_forest_feature_importances(model,X.columns)

In [ ]:
# from datanight import generate_submission
# generate_submission(preprocess_data,model,"submission.csv")

# Советы по улучшению модели:

* Смотри глазами на данные - строй графики, исследуй аномалия 
 * это может принести новые идеи и отбросить нежизнеспособные намного раньше 
 * Туториал по библиотеке, в которой можно строить графики 
   * http://matplotlib.org/users/pyplot_tutorial.html
 * Ключевой вопрос самому себе - __"От чего ещё может зависеть, сгорит ли заказ?"__


* Попробуй более точно настроить модель или выбрать другую
 * Random Forest с текущими параметрами можно улучшить
   * Документация по нему - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
 * А можно использовать другие, более подходящие методы
   * Документация по методам - http://scikit-learn.org/stable/supervised_learning.html
 * Наконец, модели можно комбинировать
   * Например, усреднять с весами
   * В итоге качество может быть лучше, чем у каждой модели по отдельности
 * Важно не увлечься - зачастую качественные изменения решения намного лучше подкручивания параметров


* Подумай, какие ещё дополнительные данные можно провязать с выборкой? 
 * насколько реально применить их за отведённое время?


* __Главное__ - пытайся понять, дадут ли твои улучшения прирост на новых данных
 * Тестовая выборка находится по времени дальше, чем обучающая.
   * Например, нельзя использовать номер дня в году, потому что мы обучаемся на выборке __ДО__ начала контрольной
 * Это не значит, что нужно отправлять миллион решений на проверку и выбирать лучшее - это приведёт к переобучению
 